In [ ]:
dbutils.widgets.text("catalog", "", "Catalog")
dbutils.widgets.text("schema", "", "Schema")

In [ ]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {schema}")

In [ ]:
%sql
CREATE TABLE if not exists crm_cust_info (
  cst_id INT,
  cst_key STRING,
  cst_firstname STRING,
  cst_lastname STRING,
  cst_marital_status STRING,
  cst_gndr STRING,
  cst_create_date DATE,
  dwh_create_date TIMESTAMP,
  audit_checksum STRING,
  is_active string,
  effective_start_date timestamp,
  effective_end_date timestamp)

In [ ]:
%sql
CREATE TABLE if not exists crm_prd_info (
  prd_id INT,
  prd_key STRING,
  prd_nm STRING,
  prd_cost INT,
  prd_line STRING,
  prd_start_dt DATE,
  prd_end_dt DATE,
  cat_id STRING,
  dwh_create_date TIMESTAMP,
  audit_checksum STRING,
  is_active string,
  effective_start_date timestamp,
  effective_end_date timestamp)

In [ ]:
%sql
CREATE TABLE if not exists crm_sales_details (
  sls_ord_num STRING,
  sls_prd_key STRING,
  sls_cust_id INT,
  sls_order_dt DATE,
  sls_ship_dt DATE,
  sls_due_dt DATE,
  sls_sales INT,
  sls_quantity INT,
  sls_price DOUBLE,
  dwh_create_date TIMESTAMP,
  audit_checksum STRING,
  is_active string,
  effective_start_date timestamp,
  effective_end_date timestamp)

In [ ]:
%sql
CREATE TABLE if not exists erp_cust_az12 (
  cid STRING,
  bdate DATE,
  gen STRING,
  dwh_create_date TIMESTAMP,
 audit_checksum STRING,
  is_active string,
  effective_start_date timestamp,
  effective_end_date timestamp)

In [ ]:
%sql
CREATE TABLE if not exists erp_loc_a101 (
  cid STRING,
  cntry STRING,
  dwh_create_date TIMESTAMP,
   audit_checksum STRING,
  is_active string,
  effective_start_date timestamp,
  effective_end_date timestamp)

In [ ]:
%sql
CREATE TABLE if not exists erp_px_cat_g1v2 (
  ID STRING,
  CAT STRING,
  SUBCAT STRING,
  MAINTENANCE STRING,
  dwh_create_date TIMESTAMP,
  audit_checksum STRING,
  is_active string,
  effective_start_date timestamp,
  effective_end_date timestamp)

In [ ]:
%sql
CREATE VIEW dim_customers AS
SELECT
    ROW_NUMBER() OVER (ORDER BY cst_id) AS customer_key, -- Surrogate key
    ci.cst_id                          AS customer_id,
    ci.cst_key                         AS customer_number,
    ci.cst_firstname                   AS first_name,
    ci.cst_lastname                    AS last_name,
    la.cntry                           AS country,
    ci.cst_marital_status              AS marital_status,
    CASE 
        WHEN ci.cst_gndr != 'n/a' THEN ci.cst_gndr -- CRM is the primary source for gender
        ELSE COALESCE(ca.gen, 'n/a')  			   -- Fallback to ERP data
    END                                AS gender,
    ca.bdate                           AS birthdate,
    ci.cst_create_date                 AS create_date
FROM gautham.gtk_scm.crm_cust_info ci
LEFT JOIN gautham.gtk_scm.erp_cust_az12 ca
    ON ci.cst_key = ca.cid and ci.is_active='Y' and ca.is_active='Y'
LEFT JOIN gautham.gtk_scm.erp_loc_a101 la
    ON ci.cst_key = la.cid and ci.is_active='Y' and la.is_active='Y';


In [ ]:
%sql
CREATE VIEW dim_products AS
SELECT
    ROW_NUMBER() OVER (ORDER BY pn.prd_start_dt, pn.prd_key) AS product_key, -- Surrogate key
    pn.prd_id       AS product_id,
    pn.prd_key      AS product_number,
    pn.prd_nm       AS product_name,
    pn.cat_id       AS category_id,
    pc.cat          AS category,
    pc.subcat       AS subcategory,
    pc.maintenance  AS maintenance,
    pn.prd_cost     AS cost,
    pn.prd_line     AS product_line,
    pn.prd_start_dt AS start_date
FROM gautham.gtk_scm.crm_prd_info pn
LEFT JOIN gautham.gtk_scm.erp_px_cat_g1v2 pc
    ON pn.cat_id = pc.id and pn.is_active='Y' and pc.is_active='Y'
WHERE pn.prd_end_dt IS NULL; -- Filter out all historical data;

In [ ]:
%sql
CREATE VIEW fact_sales AS
SELECT
    sd.sls_ord_num  AS order_number,
    pr.product_key  AS product_key,
    cu.customer_key AS customer_key,
    sd.sls_order_dt AS order_date,
    sd.sls_ship_dt  AS shipping_date,
    sd.sls_due_dt   AS due_date,
    sd.sls_sales    AS sales_amount,
    sd.sls_quantity AS quantity,
    sd.sls_price    AS price
FROM gautham.gtk_scm.crm_sales_details sd
LEFT JOIN gautham.gtk_scm.dim_products pr
    ON sd.sls_prd_key = pr.product_number and sd.is_active='Y' and pr.is_active='Y'
LEFT JOIN gautham.gtk_scm.dim_customers cu
    ON sd.sls_cust_id = cu.customer_id and sd.is_active='Y' and cu.is_active='Y';